<h2> SUBJECT 1 PREPROCESSING <h2/>

In [1]:

import mne
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

In [16]:
import mne
import numpy as np

class meg_preprocessing_pipeline:

    def __init__(self, raw):

        self.raw = raw
        self.eog_events = None
        self.ecg_events = None
        self.eog_projs = None
        self.ecg_projs = None
        self.epochs = None

    def notch_filter(self):
        self.raw.notch_filter(freqs=[50, 100, 150],
                            picks='meg',
                            method='spectrum_fit',
                            filter_length='auto',
                            fir_window='hamming',
                            fir_design='firwin2',
                            n_jobs=-1,
                            verbose=True)
        return self

    def finding_bad_channels_maxwell(self):
        """
        Using the inbuilt MNE operations to determine flat, or noisy channels as
        automatic detection which can later be interpolated and addressed.
        The automatic detection method specifically for MEG data should be
        kind enough in determining averaged cut off points, and allocating the bad annotations.

        It takes the empty list of bad channels, and updates them as it iterates over the channels.

        Returns:
            An updated list of bad channels based on noisy or flat/static channels.
            ~ Flat or static channels indicate faulty sensor.
            ~ Noisy channels indicate external noise, sensor issues etc.

        Note:
            It changes the data object in place, returning a new self.raw object
            with updated and marked bad channels.

        """
        from mne.preprocessing import find_bad_channels_maxwell
        self.raw.info['bads'] = []

        raw_check = self.raw.copy() # first uses a copy of the original raw data
        auto_noisy_chs, auto_flat_chs, auto_scores = find_bad_channels_maxwell(raw_check, verbose=True, return_scores=True)
        bads = self.raw.info['bads'] + auto_noisy_chs + auto_flat_chs # concatenates the empty list of bad channels, noisy channels, flat channels
        self.raw.info['bads'] = bads # setting the bads parameter as the concetenated list

        return self

    def interpolate_bads(self):
        """
        Interpolate bad channels in the MEG Data as marked by our bad channel detection method.

        This function firstly creates a copy of the raw data,
        then performs interpolation on the bad channels, based on good channels.

        It then assigns the interpolated data back to the original raw object.

        Returns:
            self: The updated object with interpolated bad channels.

        """
        # Create a copy of the raw data and perform interpolation
        interpolated_raw = self.raw.copy().interpolate_bads(reset_bads=True)

        # Assign interpolated data back to original raw
        self.raw = interpolated_raw
        self.raw.save("interpolated_bads_raw.fif",overwrite=True)

        return self, self.raw
    
    def estimate_continuous_head_pos(self):
        self.raw.load_data()
        chpi_freqs, ch_idx, chpi_codes = mne.chpi.get_chpi_info(info=self.raw.info)
        chpi_amplitudes = mne.chpi.compute_chpi_amplitudes(self.raw)
        chpi_locs = mne.chpi.compute_chpi_locs(self.raw.info, chpi_amplitudes)
        self.head_pos = mne.chpi.compute_head_pos(self.raw.info, chpi_locs, gof_limit=0.5, verbose=True)
        
        
        output_head_pos = 'head_pos.pos'
        mne.chpi.write_head_pos(output_head_pos, self.head_pos)
        
        return self
    
    def find_events(self):

        """
        Find and assigns annotations to events in the MEG data.

        This function uses the mapping provided in the original dataset documentation
        to assign event types based on the values detected
        in the 'STI101' stimulus channel.

        The events are then assigned annotations for later reference.

        Returns:
            self.events == an updated events object which can be used later.

        """

        # Define the mapping of event values to event types in the dict.
        mapping = {4: 'Motor/hand_imagery',
                   8: 'Motor/feet_imagery',
                   16: 'Mental/subtraction_imagery',
                   32: 'Mental/word_imagery'}

        # Find events based on the 'STI101' stimulus channel.
        all_events = mne.find_events(self.raw, stim_channel='STI101',
                                        initial_event=False,
                                        verbose=True)
        
        # Filter events to include only those specified in the mapping dictionary

        self.events = mne.pick_events(all_events, include=[4, 8, 16, 32])
        print(f"Events selected from data: {self.events[:,-1][:4]}")

        # Create annotations from the detected events, using the mapping dictionary created earlier.
        annot_from_events = mne.annotations_from_events(events=self.events,
                                                        event_desc=mapping,
                                                        sfreq=self.raw.info['sfreq'],
                                                        orig_time=self.raw.info['meas_date'])
        
        # Assign annotations to the raw data.
        self.raw.set_annotations(annot_from_events)

        # As checkpoint version control, saving events to file so we can access
        # different parts of this pipeline if we need to that also requires
        # the events file.
        mne.write_events('events.txt', self.events, overwrite=True)

        return self

    def bandpass_filter_butter(self):

        sfreq=self.raw.info['sfreq']
        nyquist_freq = sfreq / 2

        l_freq= min(7, nyquist_freq)
        h_freq= min(30, nyquist_freq)
        order=4
        ftype='butter'
        sfreq=self.raw.info['sfreq']

        iir_params = dict(order=order, ftype=ftype, output='sos')
        raw_copy = self.raw.copy()
        
        filtered_data = mne.filter.filter_data(raw_copy.get_data(), sfreq=sfreq, l_freq=l_freq, h_freq=h_freq, method='iir',
                                          phase='zero-double', iir_params=iir_params,
                                          verbose=True)
        
        self.raw = mne.io.RawArray(filtered_data, info=self.raw.info)
        self.raw.save("checkpoint_filter-raw.fif", overwrite=True)
        
        return self, self.raw

    def bandpass_butterworth_alpha_band(self):
        """
        Appling a bandpass filter to the raw data.

        This method utilizes a bandpass filter which is applied to the raw data using a bank specific for alpha frequency range {8,13}hz.

        The chosen parameters include a 'IIR' design, with a 'hamming' window.
        It utilizes the nyquist freqency ranges within both the lower and upper passband edge
        to reduce the effect of artifact aliasing.

        Returns:
            self: The modified object with filtered data.

        Note:
            This method modifies the 'raw' attribute of the object in place,
            computed across the two frequency ranges: {[8,13]} Hz.

        """

        sfreq = self.raw.info['sfreq']
        nyquist_freq = sfreq / 2

        l_freq = min(8, nyquist_freq)  # Lower cutoff frequency
        h_freq = min(13, nyquist_freq) # Upper cutoff frequency

        filter_order = 4
        ftype = 'butter'
        sfreq = self.raw.info['sfreq']
        iir_params = dict(order=filter_order, ftype=ftype)

        self.raw = self.raw.filter(l_freq=l_freq, h_freq=h_freq,
                                              method='iir', phase='zero',
                                              iir_params=iir_params,
                                              filter_length='auto',
                                              verbose=True)
        
        self.raw.save("checkpoint_filter-raw.fif", overwrite=True)
        return self


    def nyquist_st_duration(self):
        sfreq = self.raw.info['sfreq']
        nyquist_freq = sfreq / 2 # 500hz to reduce effect of aliasing

        st_duration = self.raw.times[-1] / nyquist_freq
        return st_duration
    
    def apply_tsss_filter(self):
        st_duration = self.nyquist_st_duration()
        head_pos = mne.chpi.read_head_pos(r"head_pos.pos")
        self.raw = mne.io.read_raw_fif(r"interpolated_bads_raw.fif", preload=True)
        self.raw = mne.preprocessing.maxwell_filter(self.raw, coord_frame='head', head_pos=head_pos, st_duration=st_duration, verbose=True)
        self.raw.save('tsss_checkpoint_raw.fif', overwrite=True)
        return self

    def create_eog_ecg_projs(self):

        """
        Create projs from the MEG data based on event information.

        This function uses the provided event dictionary to define event types,
        and the corresponding event codes.

        EOG/blink and ECG Artifact removal are computed via SSP projections from the MNE Library.

        Returns:
            self: ECG/EOG removed object.


        """
        self.raw = mne.io.read_raw_fif(r"tsss_checkpoint_raw.fif", preload=True)
        self.events = mne.read_events(r"events.txt")

        # Specifying the EOG and ECG channels
        eog_channel = ["EOG001", "EOG002"]
        ecg_channel = "ECG003"

        # Defining rejection criteria and flat threshold
        reject = dict(grad=4000e-13) # 4000 femtoteslas

        # Compute ECG projections
        ecg_projs, _ = mne.preprocessing.compute_proj_ecg(self.raw,
                                                                ch_name=ecg_channel,
                                                                n_grad=1,
                                                                n_mag=1,
                                                                no_proj=True)

        # Compute EOG/Blink projections
        eog_projs, _ = mne.preprocessing.compute_proj_eog(self.raw,
                                                                ch_name=eog_channel,
                                                                n_grad=1,
                                                                n_mag=1,
                                                                no_proj=True)

        # Add projectors to raw object ready for epoch creations
        self.raw.add_proj(ecg_projs)
        self.raw.add_proj(eog_projs)

        self.raw.save("tsss_eog_ecg_ssp_repaired_raw.fif", overwrite=True)

    def create_epochs(self):
        self.raw = mne.io.read_raw_fif(r"tsss_eog_ecg_ssp_repaired_raw.fif")
        # Event dictionary mapping event types to codes
        event_dict = {
            "hand_imagery": 4,
            "feet_imagery": 8,
            "subtraction_imagery": 16,
            "word_imagery": 32,
        }

        # Create epochs from raw data using events and event dict.

        """ 
        
        Whether to reject based on annotations. 
        If True (default), epochs overlapping with segments
        whose description begins with 'bad' are rejected. 
        If False, no rejection based on annotations is performed. 
        
        """

        self.epochs = mne.Epochs(self.raw, events=self.events,
                            event_id=event_dict,
                            tmin=-0.1, tmax=4, # Specifying -0.1 seconds before event onset, 4 seconds after.
                            preload=True,
                            reject_by_annotation=True, # segments marked as bad
                            baseline=None,
                            verbose=True)


        # Save epoched data to file.
        self.epochs.save('epochs-epo.fif', overwrite=True)

        return self

    

    def apply_pipeline(self):

        """
        Applies a series of preprocessing steps to the raw data based on the defined pipeline.

        Steps:
        1. Finding bad channels using Maxwell filtering.
        2. Interpolating bad channels.
        3. Applying multiple bandpass Butterworth filters.
        4. Finding events in the data.
        5. Creating epochs based on the events.

        Returns:
        -------
        self: Instance of the class.
            The modified instance of the class with the applied
            preprocessing steps

        """
        # Estimate CHP
        #self.estimate_continuous_head_pos()

        # Notch Filtering
        self.notch_filter()

        # find events
        self.find_events()
        
        # Apply a bandpass butterworth filter
        
        self.bandpass_filter_butter()
        
        # Find bad channels using maxwell filtering
        self.finding_bad_channels_maxwell()

        # bad channel interpolation
        self.interpolate_bads()
        
        # apply tsss sampling
        self.apply_tsss_filter()

        # Create projectors for blinks and heartbeats 
        self.create_eog_ecg_projs()

        # Create epochs
        self.create_epochs()

        return self


In [17]:
if __name__ == "__main__":
    import os

    while True:
        filename = input("Please specify file path: ").replace('"', '').replace("'", "")
        #filename.replace('"', '').replace("'", "")

        if os.path.isfile(filename):
            # Assuming valid file path detected
            break
        else:
            print("Invalid file path specified. Please try again.")
            break

    raw = mne.io.read_raw_fif(filename, preload=True)
    instance = meg_preprocessing_pipeline(raw)
    instance.apply_pipeline()

Opening raw data file D:\charl\Documents\CE901_MEG_DATA_AND_CODE\MEG_BIDS\MEG_BIDS\sub-1[head_movement_squid_jump_detected]\ses-1\meg\sub-1_ses-1_task-bcimici_meg.fif...
    Read a total of 13 projection items:
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
        generated with autossp-1.0.1 (1 x 306)  idle
    Range : 28000 ... 2000999 =     28.000 ...  2000.999 secs
Ready.
Reading 0 ... 1972999  =      0

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done  60 tasks      | elapsed:   42.4s
[Parallel(n_jobs=-1)]: Done 150 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 276 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 306 out of 306 | elapsed:  2.6min finished


In [ ]:
pipeline = meg_preprocessing_pipeline(sub_1_ses_1)

In [ ]:
pipeline.create_epochs()

In [ ]:
pipeline.apply_pipeline()

In [ ]:
epochs = mne.read_epochs(r"D:\charl\Documents\CE901_MEG_DATA_AND_CODE\PROCESSING_PIPE\epoched_sub_1_ses_1-epo.fif")

In [ ]:
epochs.info

In [ ]:
epochs_baseline_corrected = epochs.apply_baseline((-0.2, None))

In [ ]:
epochs_baseline_corrected.compute_psd(fmax=100).plot(picks="grad")

In [ ]:
new_sfreq = 250
epochs_resampled = epochs_baseline_corrected.copy().resample(new_sfreq, npad="auto")


In [ ]:
epochs_resampled.info

In [ ]:
epochs_resampled.compute_psd(fmax=50).plot(picks='grad')

In [ ]:
epochs_resampled.plot(picks='grad')

In [ ]:
raw = epochs_resampled

# Get the coordinates of the fiducial points
nasion = raw.info['dig'][0]['r']  # Nasion coordinate
lpa = raw.info['dig'][1]['r']  # Left preauricular point coordinate
rpa = raw.info['dig'][2]['r']  # Right preauricular point coordinate

# Calculate the mid-point between the left and right preauricular points
midpoint = (lpa + rpa)

# Calculate the vector representing the direction from the ears forward
ear_to_front_vector = nasion + midpoint

# Calculate the dot product between the vector and each channel location
channel_locations = np.array([ch['loc'][:3] for ch in raw.info['chs']])
dot_product = np.dot(channel_locations, ear_to_front_vector)

# Select channels that have positive dot product values
frontal_lobe_channels = [raw.ch_names[i] for i in np.where(dot_product > 0)[0]]


In [ ]:
print(len(frontal_lobe_channels))

In [ ]:
print(frontal_lobe_channels)
print("Number of all lobe channels forward of midpoint: ", len(frontal_lobe_channels))
# Select frontal lobe channels from MEG data
raw_frontal_lobe = raw.copy().pick_channels(frontal_lobe_channels)
raw_1 = raw_frontal_lobe.copy().pick_types('grad')
print("Number of gradiometer channels selected forward of midpoint: ", len(raw_1.info['chs']))
raw_1.compute_psd().plot()

In [ ]:
epochs_resampled.compute_psd().plot_topomap()

In [ ]:
raw_1.compute_psd().plot_topomap()

In [ ]:
raw_1.save('Sub_1_ses_1_downsampled_channel_reduction-epo.epo', overwrite=True)

In [ ]:
del sub_1_ses_1, epochs_resampled, epochs_baseline_corrected

In [ ]:
sub_1_ses_2 = mne.io.read_raw_fif("D:\charl\Documents\CE901_MEG_DATA_AND_CODE\MEG_BIDS\MEG_BIDS\sub-1\ses-2\meg\sub-1_ses-2_task-bcimici_meg.fif", preload=True, allow_maxshield=True, verbose=True)

In [ ]:
pipeline_2 = meg_preprocessing_pipeline(sub_1_ses_2)
pipeline_2.apply_pipeline()

In [ ]:
epochs_2 = mne.read_epochs(r"D:\charl\Documents\CE901_MEG_DATA_AND_CODE\PROCESSING_PIPE\SUB_1_SES_2\epoched_sub_1_ses_2-epo.fif")

In [ ]:
epochs_2.info

In [ ]:
epochs_2 = epochs_2.pick_types('grad')

epochs_2.plot()

In [ ]:
epochs_2.compute_psd().plot_topomap()

In [ ]:
epochs_2.compute_psd(fmax=60).plot()

In [ ]:
epochs_2_baseline_corrected = epochs_2.copy().apply_baseline((-0.2, None))

In [ ]:
new_sfreq = 250
epochs_2_resampled = epochs_2_baseline_corrected.copy().resample(new_sfreq, npad="auto")

In [ ]:
raw = epochs_2_resampled

# Get the coordinates of the fiducial points
nasion = raw.info['dig'][0]['r']  # Nasion coordinate
lpa = raw.info['dig'][1]['r']  # Left preauricular point coordinate
rpa = raw.info['dig'][2]['r']  # Right preauricular point coordinate

# Calculate the mid-point between the left and right preauricular points
midpoint = (lpa + rpa)

# Calculate the vector representing the direction from the ears forward
ear_to_front_vector = nasion + midpoint

# Calculate the dot product between the vector and each channel location
channel_locations = np.array([ch['loc'][:3] for ch in raw.info['chs']])
dot_product = np.dot(channel_locations, ear_to_front_vector)

# Select channels that have positive dot product values
frontal_lobe_channels = [raw.ch_names[i] for i in np.where(dot_product > 0)[0]]

In [ ]:
print(frontal_lobe_channels)
print("Number of all lobe channels forward of midpoint: ", len(frontal_lobe_channels))
# Select frontal lobe channels from MEG data
raw_frontal_lobe = raw.copy().pick_channels(frontal_lobe_channels)
raw_2 = raw_frontal_lobe.copy().pick_types('grad')
print("Number of gradiometer channels selected forward of midpoint: ", len(raw_1.info['chs']))
raw_2.compute_psd().plot()

In [ ]:
raw_2.plot_psd_topomap()

In [ ]:
raw_2.save('sub_1_ses_2_downsampled_channel_reduced-epo.epo')

In [ ]:
# Access the MEG channels and their 3D coordinates
meg_channels = [ch for ch in epochs_decimated.info['chs'] if ch['kind'] == mne.io.constants.FIFF.FIFFV_MEG_CH]
sensor_coordinates = [(ch['ch_name'], ch['loc'][:3]) for ch in meg_channels]

# Print sensor names and their 3D coordinates
for sensor_name, coord in sensor_coordinates:
    print(sensor_name, coord)

In [ ]:
epochs.plot_sensors()

In [ ]:
fiducials = [d for d in epochs.info['dig'] if d['kind'] == mne.io.constants.FIFF.FIFFV_POINT_CARDINAL]

In [ ]:
frontal_sensors = []

for sensor in sensor_locs:
    x, y, z = sensor['loc'][:3]
    if x > 0 and y > 0 and z > 0:
        frontal_sensors.append(sensor['ch_name'])

print("Frontal Lobe Sensors:")
for sensor in frontal_sensors:
    print(sensor)

In [ ]:
len(frontal_sensors)

In [ ]:
# Get the sensor locations from the info attribute
sensor_locs = epochs_decimated.info['chs']

# Create a list to store the frontal lobe sensor names
frontal_sensors = []

# Loop over the sensor locations and check for frontal lobe coordinates
for sensor in sensor_locs:
    x, y, z = sensor['loc'][:3]  # Get x, y, z coordinates
    if x > 0 and y > 0 and z > 0:
        frontal_sensors.append(sensor['ch_name'])

# Print the frontal lobe sensor names
print("Frontal Lobe Sensors:")
for sensor in frontal_sensors:
    print(sensor)

In [ ]:
len(frontal_sensors)

In [ ]:
epochs_channel_selection = epochs_decimated.copy().pick_channels(frontal_sensors)

In [ ]:
epochs_channel_selection.plot(picks='grad')

In [ ]:
epochs_channel_selection.compute_psd().plot(picks='grad')